In [1]:
#pass one image throught the system
from perceptual.filterbank import *
import os
import cv2

In [2]:
# determine what OpenCV version we are using
try:
    import cv2.cv as cv
    USE_CV2 = True
except ImportError:
    # OpenCV 3.x does not have cv2.cv submodule
    USE_CV2 = False

In [3]:
import sys
import numpy as np
import glob as gb
from pyr2arr import Pyramid2arr
from torchvision import transforms
from temporal_filters import IdealFilterWindowed, ButterBandpassFilter

In [4]:
# set path
datapaths = "../SMIC/SMIC_all_cropped/HS/*/"
action_dir = ["*/negative/*","*/positive/*","*/surprise/*","non_micro/*"]
action_names = ['negative', 'positive','surprise','non_micro-expression']
imgpath = "/*.bmp"
Logpath = "Log"
save_path = "../PhVM_Output"

steer = Steerable(5)
pyArr = Pyramid2arr(steer)

In [5]:
#load video from file
def load_video(video_name,Gwidth,Gheight):
#     print ("Reading:", video_filename)
    
    # get frame properties
    cap = gb.glob(video_name+imgpath)#open image sequence file    
    frame_count = len(cap)

    width, height,channel = cv2.imread(cap[0]).shape
    print("Gwidth,Gheight:", Gwidth,Gheight)
    if Gwidth == 0:
        Gwidth,Gheight = width, height
       
    video_tensor = np.zeros((frame_count, Gwidth, Gheight, channel),dtype='float')
    x=0
    for img in cap:
        frame = cv2.resize(cv2.imread(img), (Gheight, Gwidth), interpolation=cv2.INTER_CUBIC)
        video_tensor[x]=frame
        x+=1
    fps = 100
    
#     print ('{:d} frames'.format(frame_count))
#     print ('{:d} X {:d}'.format(width, height))
#     print ('FPS:{:d}'.format(fps))
#     print ('steer:{:d} X {:d}'.format(steer.height, steer.nbands))
    
    return video_tensor,fps, frame_count, Gwidth,Gheight

In [6]:
def save_image(final,name,s):   
    path = save_path+"/"+name+"/"+str(s)
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path) 
    ext = ".bmp"
    for i in range(len(final)):
        sample = final[i]
        filename = path+"/"+str(i)+ext
        cv2.imwrite(filename, sample)

In [7]:
# def save_image(res, nrFrames):   
#     save_name = "Images/PhaBas"
#     ext = ".png"
#     filename =save_name+ str(nrFrames)+ext
#     print(filename)
#     cv2.imwrite(filename, res)    
        

In [8]:
def phaseBasedMagnify(video_name,name,s,Gwidth,Gheight,windowSize,factor,fpsForBandPass,lowFreq,highFreq):
    print(video_name)
    #video Read
    video_tensor,fps, nrFrames,Gwidth,Gheight = load_video(video_name,Gwidth,Gheight)
    print("video_tensor", video_tensor.shape)
    final=np.zeros(video_tensor.shape)
    # read video
    # setup temporal filter
    filter = IdealFilterWindowed(windowSize, lowFreq, highFreq, fps=fpsForBandPass, outfun=lambda x: x[0])
    filter = ButterBandpassFilter(1, lowFreq, highFreq, fps=fpsForBandPass)

    for frameNr in range( nrFrames + windowSize ):
#         print (frameNr)
        sys.stdout.flush()
        if frameNr < nrFrames:
            # read frame
            im = video_tensor[frameNr]
            im = im.astype('uint8')
            if im is None:
                # if unexpected, quit
                break
#           convert to gray image
            if len(im.shape) > 2:
                 grayIm = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
            else:
                # already a grayscale image?
                grayIm = im

            # get coeffs for pyramid
            coeff = steer.buildSCFpyr(grayIm)

            # add image pyramid to video array
            # NOTE: on first frame, this will init rotating array to store the pyramid coeffs                 
            arr = pyArr.p2a(coeff)

            phases = np.angle(arr)

            # add to temporal filter
            filter.update([phases])

            # try to get filtered output to continue            
            try:
                filteredPhases = filter.next()
            except StopIteration:
                continue
            
            #motion magnification
            magnifiedPhases = (phases - filteredPhases) + filteredPhases*factor
            
            # create new array
            newArr = np.abs(arr) * np.exp(magnifiedPhases * 1j)

            # create pyramid coeffs  
            newCoeff = pyArr.a2p(newArr)
            
            # reconstruct pyramid
            out = steer.reconSCFpyr(newCoeff)

            # clip values out of range
            out[out>255] = 255
            out[out<0] = 0
            
            # make a RGB image
            rgbIm = np.empty( (out.shape[0], out.shape[1], 3 ) )
            rgbIm[:,:,0] = out
            rgbIm[:,:,1] = out
            rgbIm[:,:,2] = out
            
            #write to disk
            res = cv2.convertScaleAbs(rgbIm)
#             print('res:',res.shape)
            final[frameNr]=res
    save_image(final, name, s)
    return Gwidth,Gheight

In [9]:
# vidFname = datapaths+sample 
# the size of the sliding window
windowSize = 8
# the magnifaction factor
factor = 20
# the fps used for the bandpass
fpsForBandPass = 100 # use -1 for input video fps
# low ideal filter
lowFreq = 15
# high ideal filter
highFreq = 25
#Regulization all the image size
Gwidth, Gheight = 0,0
#Implement sample frames from each video from 4 catedories 
for fold,name in zip(action_dir,action_names):  
    fold_name = gb.glob(datapaths + fold)#open image sequence file 
    for s in range(len(fold_name)):
        video_name = fold_name[s]
        Gwidth,Gheight = phaseBasedMagnify(video_name,name,s,Gwidth,Gheight,windowSize,factor,fpsForBandPass,lowFreq,highFreq)

../SMIC/SMIC_all_cropped/HS/s11/micro/negative/s11_ne_03
Gwidth,Gheight: 0 0
video_tensor (41, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s11/micro/negative/s11_ne_02
Gwidth,Gheight: 181 148
video_tensor (31, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s11/micro/negative/s11_ne_01
Gwidth,Gheight: 181 148
video_tensor (21, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/negative/s8_ne_04
Gwidth,Gheight: 181 148
video_tensor (37, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/negative/s8_ne_03
Gwidth,Gheight: 181 148
video_tensor (28, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/negative/s8_ne_05
Gwidth,Gheight: 181 148
video_tensor (41, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/negative/s8_ne_09
Gwidth,Gheight: 181 148
video_tensor (40, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/negative/s8_ne_02
Gwidth,Gheight: 181 148
video_tensor (28, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/negative/s8_ne_07
Gwidth,Gheight: 181 148
video_tensor (23, 181, 148, 3)
../SMIC/

../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_03
Gwidth,Gheight: 181 148
video_tensor (47, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_01
Gwidth,Gheight: 181 148
video_tensor (50, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_05
Gwidth,Gheight: 181 148
video_tensor (31, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_08
Gwidth,Gheight: 181 148
video_tensor (41, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_06
Gwidth,Gheight: 181 148
video_tensor (40, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_02
Gwidth,Gheight: 181 148
video_tensor (26, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_04
Gwidth,Gheight: 181 148
video_tensor (28, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s12/micro/positive/s12_po_07
Gwidth,Gheight: 181 148
video_tensor (50, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s8/micro/positive/s8_po_02
Gwidth,Gheight: 181 148
video_tensor (28, 181, 14

../SMIC/SMIC_all_cropped/HS/s3/micro/surprise/s3_sur_03
Gwidth,Gheight: 181 148
video_tensor (21, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s20/micro/surprise/s20_sur_04
Gwidth,Gheight: 181 148
video_tensor (41, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s20/micro/surprise/s20_sur_05
Gwidth,Gheight: 181 148
video_tensor (24, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s20/micro/surprise/s20_sur_01
Gwidth,Gheight: 181 148
video_tensor (38, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s20/micro/surprise/s20_sur_02
Gwidth,Gheight: 181 148
video_tensor (22, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s20/micro/surprise/s20_sur_03
Gwidth,Gheight: 181 148
video_tensor (49, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s1/micro/surprise/s1_sur_01
Gwidth,Gheight: 181 148
video_tensor (33, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/micro/surprise/s14_sur_04
Gwidth,Gheight: 181 148
video_tensor (45, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/micro/surprise/s14_sur_05
Gwidth,Gheight: 181 148
video_tensor (41, 

../SMIC/SMIC_all_cropped/HS/s4/non_micro/s4_n10
Gwidth,Gheight: 181 148
video_tensor (21, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s6/non_micro/s6_n3
Gwidth,Gheight: 181 148
video_tensor (23, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s6/non_micro/s6_n4
Gwidth,Gheight: 181 148
video_tensor (35, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s6/non_micro/s6_n1
Gwidth,Gheight: 181 148
video_tensor (27, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s6/non_micro/s6_n2
Gwidth,Gheight: 181 148
video_tensor (24, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s19/non_micro/s19_n2
Gwidth,Gheight: 181 148
video_tensor (48, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s19/non_micro/s19_n1
Gwidth,Gheight: 181 148
video_tensor (36, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s13/non_micro/s13_n5
Gwidth,Gheight: 181 148
video_tensor (36, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s13/non_micro/s13_n9
Gwidth,Gheight: 181 148
video_tensor (40, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s13/non_micro/s13_n6
Gwidth,Gheight: 181 148


../SMIC/SMIC_all_cropped/HS/s1/non_micro/s1_n6
Gwidth,Gheight: 181 148
video_tensor (18, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s1/non_micro/s1_n4
Gwidth,Gheight: 181 148
video_tensor (24, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s1/non_micro/s1_n5
Gwidth,Gheight: 181 148
video_tensor (19, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s1/non_micro/s1_n3
Gwidth,Gheight: 181 148
video_tensor (20, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/non_micro/s14_n5
Gwidth,Gheight: 181 148
video_tensor (50, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/non_micro/s14_n6
Gwidth,Gheight: 181 148
video_tensor (36, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/non_micro/s14_n10
Gwidth,Gheight: 181 148
video_tensor (44, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/non_micro/s14_n2
Gwidth,Gheight: 181 148
video_tensor (44, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/non_micro/s14_n3
Gwidth,Gheight: 181 148
video_tensor (48, 181, 148, 3)
../SMIC/SMIC_all_cropped/HS/s14/non_micro/s14_n8
Gwidth,Gheight: 181 14